- Diabetic           yes
- Race               other
- Sex                indefinido
- GenHealth          Primero arreglar los ya definidos(Very good/Good/Excellent/Fair/Poor), los que no hacen parte del grupo 
cambiar a undefined, sacar moda y cambiar undefined a moda.
- Smoking            yes
- Stroke             yes
- BMI                promedio dependiendo del genero 



In [ ]:
import pickle as pkl
import numpy as np
import pandas as pd

In [ ]:
dataOriginal = pd.read_csv('data\heart_disease_data.csv')

In [ ]:
data = dataOriginal[pd.notnull(dataOriginal['HeartDisease'])]

In [ ]:
dataOriginal.shape

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.isnull().mean()

<font size="5"> Correción Smoking/Stroke/Heart Disease</font> 

In [ ]:
def clean_yes_no(column):
    data[column].fillna("Yes", inplace = True)
    data.loc[data[column].str.contains('Yes'), column] = "Yes"
    data.loc[data[column].str.contains('No'), column] = "No"
    data.loc[~data[column].str.contains('No'), column] = "Yes"
    

In [ ]:
dañadas = ['Smoking', 'Stroke', 'HeartDisease']
for col in dañadas:
    clean_yes_no(col)

HACER ESTO DESPUES DE HACER LAS CORRECIONES DE LOS DATOS

In [ ]:
for col in data.columns:
    if data[col].dtype == 'object':
        print(col, data[col].unique())

<font size="5"> Correción Sexo</font> 

In [ ]:
data.groupby('Sex')['Index'].nunique()

In [ ]:
data["Sex"].fillna("Other", inplace = True)
data.loc[data["Sex"].str.contains('Male'), "Sex"] = "Male"
data.loc[data["Sex"].str.contains('Female'), "Sex"] = "Female"
data.loc[~data["Sex"].str.contains('Male') & ~data["Sex"].str.contains('Female'), "Sex"] = "Other"

In [ ]:
data.groupby('Sex')['Index'].nunique()

<font size="5"> Correción Physical/ Mental health</font> 

In [ ]:
data['PhysicalHealth'].unique()

In [ ]:
def cleanNumericWithRange(column, min, max, isInteger: bool):
    data[column] = data[column].str.rstrip('abcdefghijklmnopqrstuvwxyz')
    if isInteger:
        data[column] = pd.to_numeric(data[column]).astype(int)
    else:
        data[column] = pd.to_numeric(data[column])
    data[column].clip(upper=max, lower=min, inplace=True)

In [ ]:
cleanNumericWithRange('PhysicalHealth', 0, 30, True)

In [ ]:
data['PhysicalHealth'].unique()


In [ ]:
data['MentalHealth'].unique()

In [ ]:
cleanNumericWithRange('MentalHealth', 0, 30, True)

In [ ]:
data['MentalHealth'].unique()

<font size="5"> Correción Diabetico</font> 

In [ ]:
data.groupby('Diabetic')['Index'].nunique().sort_values(ascending=False)

In [ ]:
data['Diabetic'].fillna("Yes", inplace = True)

In [ ]:
data.loc[data['Diabetic'].str.contains('Yes (during pregnancy)', regex=False), 'Diabetic'] = 'Yes (during pregnancy)'

In [ ]:
data.loc[data['Diabetic'].str.contains('No, borderline diabetes', regex=False), 'Diabetic'] = 'No, borderline diabetes'

In [ ]:
data.loc[data['Diabetic'].str.contains('No') & ~data['Diabetic'].str.contains('No, borderline diabetes'), 'Diabetic'] = 'No'

In [ ]:
data.loc[~data['Diabetic'].str.contains('No') & ~data['Diabetic'].str.contains('Yes (during pregnancy)', regex=False), 'Diabetic'] = 'Yes'

In [ ]:

data.groupby('Diabetic')['Index'].nunique().sort_values(ascending=False)

<font size="5"> Correción Raza</font> 

In [ ]:
data.groupby('Race')['Index'].nunique().sort_values(ascending=False).head(7)

In [ ]:
data['Race'].fillna('Other',inplace=True)

In [ ]:
razas = ['White','Hispanic','Black','Other','American Indian/Alaskan Native','Asian']

for raza in razas:
    data.loc[data['Race'].str.contains(raza, regex=False), 'Race'] = raza

In [ ]:
data.groupby('Race')['Index'].nunique().sort_values(ascending=False)

In [ ]:
data.loc[~(data['Race'].str.contains('White', regex=False) | data['Race'].str.contains('Hispanic', regex=False) | data['Race'].str.contains('Black', regex=False) | data['Race'].str.contains('Other', regex=False) | data['Race'].str.contains('Asian', regex=False) | data['Race'].str.contains('American Indian/Alaskan Native', regex=False)), 'Race'] = 'Other'

In [ ]:
data.groupby('Race')['Index'].nunique().sort_values(ascending=False)

<font size=5>Corrección SleepTime</font>

<p>Existen ciertos datos que no están dentro del rango aceptado >0 y <=24, sacamos un promedio de los datos que están correctos y reemplazamos los datos que están afuera del rango con el promedio calculado. Además la columna está en un tipo diferente al esperado.</p>

In [ ]:
data['SleepTime']=pd.to_numeric(data['SleepTime'])
sleepTimePromedio = data.loc[data['SleepTime'].between(0, 24, inclusive = 'left')]['SleepTime'].mean()
data.loc[~data['SleepTime'].between(0, 24, inclusive = 'left'), 'SleepTime'] = sleepTimePromedio

In [ ]:
data['SleepTime'].unique()

<font size=5>Correción AgeCategory</font>

Existen algunos datos que están dentro de un rango inferior a 50, estos datos serán cambiados por la moda de los rangos que si pertenecen a la definición.

In [ ]:
data['AgeCategory'].unique()

In [ ]:
moda = data.loc[data['AgeCategory'].str.contains('59-55') | data['AgeCategory'].str.contains('64-60') |
                data['AgeCategory'].str.contains('69-65') | data['AgeCategory'].str.contains('74-70') |
                data['AgeCategory'].str.contains('79-75') | data['AgeCategory'].str.contains('or older 80')
                ]['AgeCategory'].mode()

In [ ]:
data.loc[~(data['AgeCategory'].str.contains('59-55') | data['AgeCategory'].str.contains('64-60') |
                data['AgeCategory'].str.contains('69-65') | data['AgeCategory'].str.contains('74-70') |
                data['AgeCategory'].str.contains('79-75') | data['AgeCategory'].str.contains('or older 80'))
        , 'AgeCategory'] = moda[0]

<font size=5>Correción BMI</font>

In [ ]:
copia = data[['Sex','BMI']].copy()
copia[pd.notnull(copia['BMI'])]
copia['BMI'] = copia['BMI'].str.rstrip('abcdefghijklmnopqrstuvwxyz')
copia['BMI'] = pd.to_numeric(copia['BMI']).astype(float)
copia.drop(copia[copia['BMI'] >=99].index, inplace = True)
copia.drop(copia[copia['BMI'] <=1].index, inplace = True)
promedios = copia.groupby('Sex')['BMI'].mean()

In [ ]:
pd.options.mode.chained_assignment = None
data["BMI"].fillna(100, inplace = True)
data["BMI"] = data["BMI"].str.rstrip('abcdefghijklmnopqrstuvwxyz')
data["BMI"].fillna(100, inplace = True)

In [ ]:

data['BMI'] = pd.to_numeric(data['BMI']).astype(float)
data.loc[(data["BMI"].gt(98) & data["Sex"].str.contains('Female')), "BMI"] = round(promedios.Female,2)
data.loc[(data["BMI"].gt(98) & data["Sex"].str.contains('Male')), "BMI"] = round(promedios.Male,2)
data.loc[(data["BMI"].gt(98) & data["Sex"].str.contains('Other')), "BMI"] = round(promedios.Other,2)

In [ ]:
np.set_printoptions(threshold=np.inf)
data['BMI'].describe()


In [ ]:
data['BMI'].unique()

<font size=5>Correción GenHealth</font>

Se corrigieron los valores que tenían los valores correctos pero letras adcionales, tambien se reemplazaron los nulos y los valores incorrectos por la moda.

In [ ]:
data['GenHealth'].fillna('None', inplace = True)
data.loc[data['GenHealth'].str.contains('Very good'), 'GenHealth'] = 'Very good'
data.loc[data['GenHealth'].str.contains('Good'), 'GenHealth'] = 'Good'
data.loc[data['GenHealth'].str.contains('Excellent'), 'GenHealth'] = 'Excellent'
data.loc[data['GenHealth'].str.contains('Fair'), 'GenHealth'] = 'Fair'
data.loc[data['GenHealth'].str.contains('Poor'), 'GenHealth'] = 'Poor'
mode = data['GenHealth'].mode()

In [ ]:
data.loc[~(data['GenHealth'].str.contains('Very good')) & ~(data['GenHealth'].str.contains('Good')) & ~(data['GenHealth'].str.contains('Excellent')) & ~(data['GenHealth'].str.contains('Fair')) & ~(data['GenHealth'].str.contains('Poor')), 'GenHealth'] = mode[0]

<font size=5>Convertir datos limpios a csv</font>

In [ ]:
data.to_csv(path_or_buf='datos_convertidos.csv',index=False)